In [1]:
using Revise

# A tutorial for the package ```ChebyshevFiltering```.

In this notebook we will provide a tutorial on how to use the the package ```ChebyshevFiltering``` for sparse diagonalization of Hermitian matrices (both real and complex).

Let us first load the package

In [2]:
using ChebyshevFiltering
include("/home/dario/Dropbox/delocalization_and_growth/revision_chaos_paper_code/QMB_module.jl")
using .ImpurityUtilities

and now let's create an SYK-like Hamiltonian. We also renormalize its spectrum to lie in the interval $\left[-1 , \, 1 \right]$/

In [3]:
const number_of_majorana = 20
majorana_matrices = majorana_matrices_chiral(number_of_majorana)
hamiltonian_matrix, edges_graph, weights_graph = hamiltonian_chiral(number_of_majorana, majorana_matrices, 4, 1.0)
ChebyshevFiltering.renormalization_hamiltonian!(hamiltonian_matrix)

512×512 SparseArrays.SparseMatrixCSC{ComplexF64, Int64} with 15360 stored entries:
⎡⢿⣷⣿⣼⠿⠷⢄⢸⣾⡿⠇⠀⠀⠀⠀⠳⣄⢠⣼⠿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠈⠳⣄⢹⣶⡟⎤
⎢⣛⣿⣿⣿⣴⣦⣾⣿⢟⠙⠂⠀⠀⠀⠀⣀⣾⡿⣟⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠐⣦⣼⠿⣍⠙⎥
⎢⢿⡇⠰⣿⣿⣿⣟⣦⢨⣷⣄⠀⠀⢠⣤⡿⠇⠀⠈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣄⠀⢤⣈⣷⣟⠛⠆⠈⠳⎥
⎢⣀⣑⣾⣿⠻⣽⣿⣿⣿⢉⡉⣳⣼⣿⠉⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡈⢳⣤⠛⢧⠈⠳⣄⠀⠀⎥
⎢⣾⡿⣟⠑⢦⣶⡟⢛⣿⣿⣷⡟⠛⠓⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡹⢿⡉⠳⣄⠀⠀⠈⠳⣄⎥
⎢⠉⠁⠈⠀⠀⠙⢧⣨⣽⠿⣿⣿⣿⣼⠿⠗⣄⢸⣾⡿⠳⣄⠀⠀⠈⠳⣄⣙⣶⡟⠁⠀⠁⠀⠈⠀⠀⠀⠀⠈⎥
⎢⠀⠀⠀⠀⠀⣀⣶⣿⢿⠀⣛⣿⣿⣿⣶⣦⣶⡿⣟⠙⠀⠈⠳⣄⠐⢦⣼⠿⣇⠙⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⢤⡀⠀⢠⣤⡿⠇⠀⠀⠙⢿⠇⠸⣿⣿⣿⣯⡆⢨⣷⢄⠀⢠⣈⣿⣿⠙⠆⠈⠳⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⣙⣾⡿⠉⠁⠀⠀⠀⠀⣀⣙⣼⡿⠫⠿⣿⣿⣿⣉⡈⢱⣴⠟⠧⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣶⡟⠛⠙⢦⡀⠀⠀⠀⠀⣾⡿⣟⠙⢦⣶⡟⢻⣿⣿⣼⠿⡉⠱⢄⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠑⢆⣈⣶⡟⣿⣿⣧⣼⠿⠳⣄⣽⣾⡿⠀⠀⠀⠀⠈⠳⣄⣤⣼⠿⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⢲⣴⠟⢇⡈⣉⣿⣿⣿⣶⣢⣾⡟⣍⠉⠀⠀⠀⠀⢀⣀⣾⡿⣍⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡀⠰⣄⣿⣿⡉⠃⠀⠑⢿⡃⠸⣻⣿⣿⣿⡆⢰⣷⣄⠀⠀⢰⣾⠛⠃⠀⠈⠓⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣄⢹⣶⡟⠳⠄⠙⢦⡀⠀⣄⣽⣾⠿⠻⠿⣿⣿⣿⣭⠀⣷⣿⠿⠉⠀⠀⠀⠀⠀⎥
⎢⡀⠀⠀⠀⠀⡀⠀⢀⠀⢀⣼⠿⣍⠙⢦⡀⠀⠀⠙⢦⣾⡿⡇⠙⢴⣶⡟⣿⣿⣿⣶⣟⡋⢳⣄⠀⠀⡀⢀⣀⎥
⎢⠙⢦⡀⠀⠀⠙⢦⣈⣷⣎⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢤⣤⣼⢿⣿⣿⣥⣼⠿⠳⢄⣽⣾⡿⎥
⎢⠀⠀⠙⢦⡀⢳⣤⠛⢧⡈⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣿⡟⢯⣈⣁⣿⣿⣿⣟⣦⣿⡿⢍⠉⎥
⎢⢦⡀⠰⣤⣽⢿⡉⠓⠀⠙⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⢦⡀⠀⢰⣾⠛⠃⠀⠀⠙⢿⡃⠻⣽⣿⣿⣿⠆⢸⣷⎥
⎢⣄⣙⣶⡟⠻⠄⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣽⣾⡿⠉⠀⠀⠀⠀⠠⣄⣵⣿⡿⠻⠟⣿⣿⣿⣭⎥
⎣⣼⠿⣇⠙⢦⡀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣶⡟⠃⠙⢦⠀⠀⠀⠀⢰⣾⡿⡇⠑⢶⣶⡟⣿⢿⣷⎦

## The estimate of the density of states via Kernel Polynomial Method.

We now test how works the estimate of the density of states, computed by means of the Kernel Polynomial Method.

We start by fixing the relevant parameters, and the window of the spectrum where we want to estimate the density

In [4]:
max_KPM_degree = 150
stochastic_dimension = 30
lambda_min = -0.1
lambda_max = 0.1

0.1

and now we can compute the approximated density and compute the number of expected states

In [6]:
approximated_density = KPM_density(hamiltonian_matrix, max_KPM_degree, stochastic_dimension)
x_values = lambda_min:((lambda_max - lambda_min) / 200):lambda_max
y_values = [approximated_density(x) for x in x_values]

using NumericalIntegration
expected_value = NumericalIntegration.integrate(x_values, y_values)

101.0318528486316

which we compare against the actual value:

In [7]:
using LinearAlgebra
real_number = length([value  for value in eigvals(Matrix(hamiltonian_matrix)) if lambda_min < value < lambda_max])

101

We see that the agreement is excellent.